<a href="https://colab.research.google.com/github/monarkparekh/MAE-598__Design-Optimization/blob/Assignment-5/MAE_598__Design_Optimization_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><h1><center>MAE 598 Design Optimization: 
Assignment 5</center></h1></b>

<h3><center>Name: Monark Parekh <br>
ASU ID: 1222179426</center></h3>